In this notebook I describe how to decompose matrices.

In [55]:
from sympy import *
from sympy.physics.quantum.dagger import Dagger

varphi = Symbol('varphi')
phi = Symbol('phi')
theta = Symbol('theta')
lambda_p = Symbol('lambda')

varphi, theta, phi, lambda_p, pi

(varphi, theta, phi, lambda, pi)

In [56]:
import numpy as np

In [57]:
def Z01(varphi):
    return Matrix([
        [exp(-I*varphi), 0, 0],
        [0, 1, 0],
        [0, 0, 1]
    ])

def Z12(varphi):
    return Matrix([
        [1, 0, 0],
        [0, 1, 0],
        [0, 0, exp(I*varphi)]
    ])

def X01(theta):
    return Matrix([
        [cos(theta/2), -I*sin(theta/2), 0],
        [-I*sin(theta/2), cos(theta/2), 0],
        [0, 0, 1]
    ])

def X12(theta):
    return Matrix([
        [1, 0, 0],
        [0, cos(theta/2), -I*sin(theta/2)],
        [0, -I*sin(theta/2), cos(theta/2)],
    ])

def R01(theta, phi, lambda_p):
    return Matrix([
        [cos(theta/2), -I*exp(-I*phi)*sin(theta/2), 0],
        [-I*exp(I*phi)*sin(theta/2), exp(I*(lambda_p+phi))*cos(theta/2), 0],
        [0, 0, 1]
    ])

def R12(theta, phi, lambda_p):
    return Matrix([
        [1, 0, 0],
        [0, cos(theta/2), -I*exp(I*lambda_p)*sin(theta/2)],
        [0, -I*exp(I*phi)*sin(theta/2), exp(I*(lambda_p+phi))*cos(theta/2)]
    ])

def ket0():
    return Matrix([
        [1], [0], [0]
    ])

def ket1():
    return Matrix([
        [0], [1], [0]
    ])

def ket2():
    return Matrix([
        [0], [0], [1]
    ])

In [58]:
# American way of factorizing: you pull out the exact amount you specify in factor.
# For example, if you want to pull out exp(i*pi). Then factor is pi.
# If you want to pull out exp(-ipi/2). Then your factor is -pi/2

def factorize(factor, matrix, dim):
    ID = eye(dim)
    ID = exp(-I*factor)*ID
    return simplify(cancel(ID*matrix))

# British way of factorizing: you pull out the negated amount you specify in factor.
# For example, if you want to pull out exp(i*pi). Then factor is -pi.
# If you want to pull out exp(-ipi/2). Then your factor is +pi/2

def factorise(factor, matrix, dim):
    ID = eye(dim)
    ID = exp(I*factor)*ID
    return simplify(cancel(ID*matrix))

---

# Facto 1. Any SU(2) gate can be written in the form of a 2x2 matrix with three parameters

In [319]:
# This is the original one

U3(theta, phi, lambda_p)

Matrix([
[              cos(theta/2),      -I*exp(I*lambda)*sin(theta/2)],
[-I*exp(I*phi)*sin(theta/2), exp(I*(lambda + phi))*cos(theta/2)]])

In [320]:
# This is the decomposed one

Z2x2(phi)*X2x2(theta)*Z2x2(lambda_p)

Matrix([
[  exp(-I*lambda/2)*exp(-I*phi/2)*cos(theta/2), -I*exp(I*lambda/2)*exp(-I*phi/2)*sin(theta/2)],
[-I*exp(-I*lambda/2)*exp(I*phi/2)*sin(theta/2),     exp(I*lambda/2)*exp(I*phi/2)*cos(theta/2)]])

In [321]:
# They are equivalent, up to a global phase of -(lambda+phi)/2

factorize(-(lambda_p+phi)/2, Z2x2(phi)*X2x2(theta)*Z2x2(lambda_p), 2)

Matrix([
[              cos(theta/2),      -I*exp(I*lambda)*sin(theta/2)],
[-I*exp(I*phi)*sin(theta/2), exp(I*(lambda + phi))*cos(theta/2)]])

# Facto 2. $R^{01}(\theta, \phi, \lambda)= \mathbf{Z}^{12}(-\lambda-\phi) Z^{01}(\phi)X^{01}(\theta)Z^{01}(\lambda)$

In [322]:
# The original one

R01(theta, phi, lambda_p)

Matrix([
[              cos(theta/2),      -I*exp(I*lambda)*sin(theta/2), 0],
[-I*exp(I*phi)*sin(theta/2), exp(I*(lambda + phi))*cos(theta/2), 0],
[                         0,                                  0, 1]])

In [323]:
# The decomposed one, without an additional Z12 gate

simplify(Z01(phi)*X01(theta)*Z01(lambda_p))

Matrix([
[exp(-I*(lambda + phi))*cos(theta/2), -I*exp(-I*phi)*sin(theta/2), 0],
[     -I*exp(-I*lambda)*sin(theta/2),                cos(theta/2), 0],
[                                  0,                           0, 1]])

In [324]:
# Now we include a Z12 gate

simplify(Z12(-lambda_p-phi)*Z01(phi)*X01(theta)*Z01(lambda_p))

Matrix([
[exp(-I*(lambda + phi))*cos(theta/2), -I*exp(-I*phi)*sin(theta/2),                      0],
[     -I*exp(-I*lambda)*sin(theta/2),                cos(theta/2),                      0],
[                                  0,                           0, exp(-I*(lambda + phi))]])

In [325]:
# The above sequence is equivalent to the original one, up to a global phase of -lambda-phi

factorize(-lambda_p-phi, Z12(-lambda_p-phi)*Z01(phi)*X01(theta)*Z01(lambda_p), 3)

Matrix([
[              cos(theta/2),      -I*exp(I*lambda)*sin(theta/2), 0],
[-I*exp(I*phi)*sin(theta/2), exp(I*(lambda + phi))*cos(theta/2), 0],
[                         0,                                  0, 1]])

# Facto 3. $X^{01}(\theta)= \mathbf{Z}^{12}(\theta/2)Z^{01}(-\pi/2)SX^{01}Z^{01}(\pi-\theta)SX^{01}Z^{01}(-\pi/2)$

In [326]:
# The original one 

X01(theta)

Matrix([
[   cos(theta/2), -I*sin(theta/2), 0],
[-I*sin(theta/2),    cos(theta/2), 0],
[              0,               0, 1]])

In [327]:
# The decomposed one, without an additional Z12 gate

simplify(Z01(-pi/2)*X01(pi/2)*Z01(pi-theta)*X01(pi/2)*Z01(-pi/2))

Matrix([
[exp(I*theta)/2 + 1/2, 1/2 - exp(I*theta)/2, 0],
[1/2 - exp(I*theta)/2, exp(I*theta)/2 + 1/2, 0],
[                   0,                    0, 1]])

### Note that $(1+e^{i\theta})/2=e^{i\theta/2}(e^{i\theta/2}+e^{-i\theta/2})/2= e^{i\theta/2}\cos(\theta/2)$ and similarly for $(1-e^{i\theta})/2=-ie^{i\theta/2}\sin(\theta/2)$

In [328]:
# The decomposed one, with an additional Z12 gate

simplify(Z12(theta/2)*Z01(-pi/2)*X01(pi/2)*Z01(pi-theta)*X01(pi/2)*Z01(-pi/2))

Matrix([
[exp(I*theta)/2 + 1/2, 1/2 - exp(I*theta)/2,              0],
[1/2 - exp(I*theta)/2, exp(I*theta)/2 + 1/2,              0],
[                   0,                    0, exp(I*theta/2)]])

In [329]:
# The above one is equivalent to the original one, up to a global phase of theta/2.

factorize(theta/2, Z12(theta/2)*Z01(-pi/2)*X01(pi/2)*Z01(pi-theta)*X01(pi/2)*Z01(-pi/2), 3)

Matrix([
[   cos(theta/2), -I*sin(theta/2), 0],
[-I*sin(theta/2),    cos(theta/2), 0],
[              0,               0, 1]])

Upon combining Facto 2 and 3, we have 

# Fact 4. $R^{01}(\theta, \phi, \lambda)=\mathbf{Z}^{12}(-\lambda-\phi+\theta/2) Z^{01}(\phi-\pi/2)SX^{01}Z^{01}(\pi-\theta)SX^{01}Z^{01}(\lambda-\pi/2)$

In [330]:
# The original one

R01(theta, phi, lambda_p)

Matrix([
[              cos(theta/2),      -I*exp(I*lambda)*sin(theta/2), 0],
[-I*exp(I*phi)*sin(theta/2), exp(I*(lambda + phi))*cos(theta/2), 0],
[                         0,                                  0, 1]])

In [331]:
# The decomposed one

simplify(Z12(-lambda_p-phi+theta/2)*Z01(phi-pi/2)*X01(pi/2)*Z01(pi-theta)*X01(pi/2)*Z01(lambda_p-pi/2))

Matrix([
[(exp(I*(theta - pi)) - 1)*exp(-I*(lambda + phi - pi))/2, (1 - exp(I*theta))*exp(-I*phi)/2,                                    0],
[                    (1 - exp(I*theta))*exp(-I*lambda)/2,             exp(I*theta)/2 + 1/2,                                    0],
[                                                      0,                                0, exp(-I*(2*lambda + 2*phi - theta)/2)]])

In [332]:
# The above one is equivalent to the original one, up to a global phase of lambda+phi-theta/2

factorize(-lambda_p-phi+theta/2, Z12(-lambda_p-phi+theta/2)*Z01(phi-pi/2)*X01(pi/2)*Z01(pi-theta)*X01(pi/2)*Z01(lambda_p-pi/2), 3)

Matrix([
[                                         cos(theta/2),               (exp(I*lambda) - exp(I*(lambda + theta)))*exp(-I*theta/2)/2, 0],
[(exp(I*phi) - exp(I*(phi + theta)))*exp(-I*theta/2)/2, (exp(I*(lambda + phi)) + exp(I*(lambda + phi + theta)))*exp(-I*theta/2)/2, 0],
[                                                    0,                                                                         0, 1]])

## Matrix element [0, 0] is ok. For matrix element [1,1].

## \begin{align}
M[1,1] &= e^{i(\lambda+\phi)}(1+e^{i\theta})e^{-i\theta/2}/2\\
&=e^{i(\lambda+\phi)}(e^{-i\theta/2}+e^{i\theta/2})/2\\
&=e^{i(\lambda+\phi)}\cos(\theta/2)
\end{align}

## For matrix element [0, 1].

## \begin{align} 
M[0,1] &= e^{i\lambda}(1-e^{i\theta})e^{-i\theta/2}/2\\
&=e^{i\lambda}(e^{-i\theta/2}-e^{i\theta/2})/2\\
&=-ie^{i\lambda}\sin(\theta/2)
\end{align}

## Similarly for matrix element [1,0].

## \begin{align} 
M[0,1] &= e^{i\phi}(1-e^{i\theta})e^{-i\theta/2}/2\\
&=e^{i\phi}(e^{-i\theta/2}-e^{i\theta/2})/2\\
&=-ie^{i\phi}\sin(\theta/2)
\end{align}

## Hence Facto 4. $R^{01}(\theta, \phi, \lambda)=\mathbf{Z}^{12}(-\lambda-\phi+\theta/2) Z^{01}(\phi-\pi/2)SX^{01}Z^{01}(\pi-\theta)SX^{01}Z^{01}(\lambda-\pi/2)$

# Now I redo the above steps, only to find a similar decomposition for $(12)$ subspace. The origin of these $Z^{12}$ gates in $(01)$ decomposition shall be discussed later.

Let us follow a heuristic approach. 

# Facto 5. $R^{12}(\theta, \phi, \lambda)= Z^{12}(\phi)X^{12}(\theta)Z^{12}(\lambda)$

In [333]:
# The original one

R12(theta, phi, lambda_p)

Matrix([
[1,                          0,                                  0],
[0,               cos(theta/2),      -I*exp(I*lambda)*sin(theta/2)],
[0, -I*exp(I*phi)*sin(theta/2), exp(I*(lambda + phi))*cos(theta/2)]])

In [334]:
# The decomposed one, without an additional Z01 gate

simplify(Z12(phi)*X12(theta)*Z12(lambda_p))

Matrix([
[1,                          0,                                  0],
[0,               cos(theta/2),      -I*exp(I*lambda)*sin(theta/2)],
[0, -I*exp(I*phi)*sin(theta/2), exp(I*(lambda + phi))*cos(theta/2)]])

# What!?

# Facto 6. $X^{12}(\theta)= Z^{01}(\theta/2)Z^{12}(-\pi/2)SX^{12}Z^{12}(\pi-\theta)SX^{12}Z^{12}(-\pi/2)$

In [299]:
# The original one

X12(theta) 

Matrix([
[1,               0,               0],
[0,    cos(theta/2), -I*sin(theta/2)],
[0, -I*sin(theta/2),    cos(theta/2)]])

In [301]:
# The decomposed one, without an additional Z01 gate

simplify(Z12(-pi/2)*X12(pi/2)*Z12(pi-theta)*X12(pi/2)*Z12(-pi/2))

Matrix([
[1,                      0,                      0],
[0,  1/2 + exp(-I*theta)/2, -1/2 + exp(-I*theta)/2],
[0, -1/2 + exp(-I*theta)/2,  1/2 + exp(-I*theta)/2]])

## $1/2(1+e^{-i\theta})= e^{-i\theta/2}\cos(\theta/2)$ and $-1/2(1-e^{-i\theta})=-ie^{-i\theta/2}\sin(\theta/2)$.

In [305]:
# The decomposed one, with an additional Z01 gate to compensate for the additonal phase.
# It is equivalent to the original one, up to a global phase of -theta/2.

factorize(-theta/2, Z01(theta/2)*Z12(-pi/2)*X12(pi/2)*Z12(pi-theta)*X12(pi/2)*Z12(-pi/2), 3)

Matrix([
[1,               0,               0],
[0,    cos(theta/2), -I*sin(theta/2)],
[0, -I*sin(theta/2),    cos(theta/2)]])

Upon comibing Facto 5 and 6, we have 

\begin{align}
R^{12}(\theta, \phi, \lambda) &= Z^{12}(\phi)X^{12}(\theta)Z^{12}(\lambda)\\
&= Z^{12}(\phi)Z^{01}(\theta/2)Z^{12}(-\pi/2)SX^{12}Z^{12}(\pi-\theta)SX^{12}Z^{12}(-\pi/2)Z^{12}(\lambda)
\end{align}

# Facto 7. $R^{12}(\theta, \phi, \lambda)=\mathbf{Z}^{01}(\theta/2) Z^{12}(\phi-\pi/2)SX^{12}Z^{12}(\pi-\theta)SX^{12}Z^{12}(\lambda-\pi/2)$

In [306]:
# The original one

R12(theta, phi, lambda_p)

Matrix([
[1,                          0,                                  0],
[0,               cos(theta/2),      -I*exp(I*lambda)*sin(theta/2)],
[0, -I*exp(I*phi)*sin(theta/2), exp(I*(lambda + phi))*cos(theta/2)]])

In [307]:
# The decomposed one, with an additional Z01 rotation

Z01(theta/2)*Z12(phi-pi/2)*X12(pi/2)*Z12(pi-theta)*X12(pi/2)*Z12(lambda_p-pi/2)

Matrix([
[exp(-I*theta/2),                                                                      0,                                                                                          0],
[              0,                                            1/2 - exp(I*(pi - theta))/2,                                    (-I*exp(I*(pi - theta))/2 - I/2)*exp(I*(lambda - pi/2))],
[              0, -I*exp(I*(pi - theta))*exp(I*(phi - pi/2))/2 - I*exp(I*(phi - pi/2))/2, (exp(I*(pi - theta))*exp(I*(phi - pi/2))/2 - exp(I*(phi - pi/2))/2)*exp(I*(lambda - pi/2))]])

In [309]:
# They are equivalent, up to a global phase of -theta/2

factorize(-theta/2, Z01(theta/2)*Z12(phi-pi/2)*X12(pi/2)*Z12(pi-theta)*X12(pi/2)*Z12(lambda_p-pi/2), 3)

Matrix([
[1,                                           0,                                                    0],
[0,                                cos(theta/2),       (1 - exp(I*theta))*exp(I*(lambda - theta/2))/2],
[0, (1 - exp(I*theta))*exp(I*(phi - theta/2))/2, (exp(I*theta) + 1)*exp(I*(lambda + phi - theta/2))/2]])

## Matrix element [1, 1] is ok. For matrix element [2, 2].

## \begin{align}
M[2, 2] &= e^{i(\lambda+\phi)}(1+e^{i\theta})e^{-i\theta/2}/2\\
&=e^{i(\lambda+\phi)}(e^{-i\theta/2}+e^{i\theta/2})/2\\
&=e^{i(\lambda+\phi)}\cos(\theta/2)
\end{align}

## For matrix element [1, 2].

## \begin{align} 
M[1,2] &= e^{i\lambda}(1-e^{i\theta})e^{-i\theta/2}/2\\
&=e^{i\lambda}(e^{-i\theta/2}-e^{i\theta/2})/2\\
&=-ie^{i\lambda}\sin(\theta/2)
\end{align}

## Similarly for matrix element [2, 1].

## \begin{align} 
M[2,1] &= e^{i\phi}(1-e^{i\theta})e^{-i\theta/2}/2\\
&=e^{i\phi}(e^{-i\theta/2}-e^{i\theta/2})/2\\
&=-ie^{i\phi}\sin(\theta/2)
\end{align}

---

# Geometric Berry phase

In [89]:
def R01(phi, theta):
    return Matrix([
        [cos(theta/2), -I*exp(-I*phi)*sin(theta/2), 0],
        [-I*exp(I*phi)*sin(theta/2), cos(theta/2), 0],
        [0, 0, 1]
    ])

def R12(phi, theta):
    return Matrix([
        [1, 0, 0],
        [0, cos(theta/2), -I*exp(-I*phi)*sin(theta/2)],
        [0, -I*exp(I*phi)*sin(theta/2), cos(theta/2)]
    ])

In [90]:
def ket0():
    return Matrix([
        [1], [0], [0]
    ])

In [91]:
Dagger(R12(pi/2,pi/4)) * R12(pi/2,pi/4)

Matrix([
[1, 0, 0],
[0, 1, 0],
[0, 0, 1]])

In [92]:
state = ket0()
state = R01(0, pi/2) * state

In [93]:
state

Matrix([
[   sqrt(2)/2],
[-sqrt(2)*I/2],
[           0]])

In [94]:
state = R12(0, pi) * state
# state = R12(pi/4, pi) * state

state

Matrix([
[ sqrt(2)/2],
[         0],
[-sqrt(2)/2]])

In [95]:
state = R12(phi, pi) * state

state

Matrix([
[              sqrt(2)/2],
[sqrt(2)*I*exp(-I*phi)/2],
[                      0]])

In [96]:
state = R01(varphi, pi/2) * state

state

Matrix([
[  1/2 + exp(-I*phi)*exp(-I*varphi)/2],
[-I*exp(I*varphi)/2 + I*exp(-I*phi)/2],
[                                   0]])

In [98]:
simplify(Dagger(state) * state)

Matrix([[(-(1 - exp(I*(phi + varphi)))*(1 - exp(I*(conjugate(phi) + conjugate(varphi))))*exp(I*(phi + varphi)) + (exp(I*(phi + varphi)) + 1)*(exp(I*(conjugate(phi) + conjugate(varphi))) + 1)*exp(I*(phi + conjugate(varphi))))*exp(-I*(2*phi + varphi + conjugate(varphi)))/4]])

In [44]:
population = []
varphi_list = [n*2*pi for n in np.linspace(0, 1, 10)]

for varphi in varphi_list:
    state = R01(varphi, pi/2) * state
    pop0 = abs((Dagger(ket0()) * state)[0])**2
    pop1 = abs((Dagger(ket1()) * state)[0])**2
    pop2 = abs((Dagger(ket2()) * state)[0])**2
    population.append((pop0, pop1, pop2))

KeyboardInterrupt: 

In [38]:
varphi_list = [n*2*pi for n in np.linspace(0, 1, 100)]